In [1]:
import os
import cv2

In [2]:
base_url = "datasets"
train_data = []
train_label = []

In [3]:
for i in [0,2,3,4,5,6,7,8,9]:
    folder_url = base_url + "/" + str(i)
    for r, d, f in os.walk(folder_url):
        for file in f:
            img = cv2.imread(os.path.join(r, file),0)
            train_data.append(img)
            train_label.append(i)

In [4]:
len(train_data)

1423

In [5]:
len(train_label)

1423

In [6]:
import random
import numpy as np

In [7]:
shuffle_order = list(range(1200))
random.shuffle(shuffle_order)

train_data = np.array(train_data)
train_label = np.array(train_label)

train_data = train_data[shuffle_order]
train_label = train_label[shuffle_order]

In [8]:
train_x = train_data[:700]
train_y = train_label[:700]

val_x = train_data[700:1000]
val_y = train_label[700:1000]

test_x = train_data[1000:]
test_y = train_label[1000:]

In [9]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers import regression
from tflearn.data_utils import to_categorical

W0722 23:53:06.086357 140068096689920 deprecation_wrapper.py:119] From /home/phamhung/Downloads/handle-image/venv/lib/python3.5/site-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0722 23:53:06.087943 140068096689920 deprecation_wrapper.py:119] From /home/phamhung/Downloads/handle-image/venv/lib/python3.5/site-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0722 23:53:06.096041 140068096689920 deprecation_wrapper.py:119] From /home/phamhung/Downloads/handle-image/venv/lib/python3.5/site-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0722 23:53:06.101427 140068096689920 deprecation_wrapper.py:119] From /home/phamhung/Downloads/handle-image/venv/lib/python3.5/site-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please 

Scipy not supported!


In [10]:
BATCH_SIZE = 32
IMG_SIZE = 75
N_CLASSES = 9
LR = 0.001
N_EPOCHS = 50

In [11]:
tf.reset_default_graph()

network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1]) #1

network = conv_2d(network, 32, 3, activation='relu') #2
network = max_pool_2d(network, 2) #3

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = fully_connected(network, 1024, activation='relu') #4
network = dropout(network, 0.8) #5

network = fully_connected(network, N_CLASSES, activation='softmax')#6
network = regression(network)

model = tflearn.DNN(network) #7

W0722 23:53:06.179690 140068096689920 deprecation_wrapper.py:119] From /home/phamhung/Downloads/handle-image/venv/lib/python3.5/site-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 23:53:06.184627 140068096689920 deprecation.py:506] From /home/phamhung/Downloads/handle-image/venv/lib/python3.5/site-packages/tflearn/initializations.py:119: calling UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0722 23:53:06.185550 140068096689920 deprecation.py:323] From /home/phamhung/Downloads/handle-image/venv/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:507: UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) is deprecated and will be removed in a future version.
Instructions for updatin

In [12]:
train_x = train_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
val_x = val_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_x = test_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [13]:
original_test_y = test_y # được sử dụng để test ở bước sau

train_y = to_categorical(train_y, N_CLASSES)
val_y = to_categorical(val_y, N_CLASSES)
test_y = to_categorical(test_y, N_CLASSES)


In [14]:
model.fit(train_x, train_y, n_epoch=N_EPOCHS, validation_set=(val_x, val_y), show_metric=True)
model.save('mymodel.tflearn')

Training Step: 549  | total loss: 0.24070 | time: 6.040s
| Adam | epoch: 050 | loss: 0.24070 - acc: 0.9795 -- iter: 640/700
Training Step: 550  | total loss: 0.21696 | time: 7.568s
| Adam | epoch: 050 | loss: 0.21696 - acc: 0.9815 | val_loss: 0.28598 - val_acc: 0.9467 -- iter: 700/700
--


In [20]:
model.load('mymodel.tflearn')
img = cv2.imread("datasets/6/20491.png",0)
img = img.reshape(-1, 75, 75, 1)
result = model.predict(img)
result = np.argmax(result, axis=-1)
print(result)

[6]
